# ラベル、非ニューラルネットワークモデル

処理の流れ

正解ラベルの作成 → 学習データと検証データに分割 → 学習

前項で作成した文書ベクトル（拡張子は.mtx）を学習データ（x_train）と検証データ（x_test）に分割します。

正解ラベルも学習データ（y_train）と検証データ（y_test）に分割します。

まず学習データを用いて学習させ、得られた機械学習モデルに検証データを入れて出力（pred_data）を作り、検証データ（y_test）と照合することで精度を求めます。

ここではSVMによる2値分類とXGBoostによる4値分類を行います。


In [1]:
import os
from google.colab import drive

# googleドライブのマウント 
drive.mount('/content/drive/')

# 作業ファイルをマイドライブに変更
os.chdir('/content/drive/My Drive/') 

Mounted at /content/drive/


In [2]:
# pandasモジュールを用いたサンプルデータの読み込み

import pandas as pd

data = pd.read_excel('data/sample_data.xlsx')
data.head()

,公報番号(全文リンク),発明等の名称,出願人・権利者(最新),ＩＰＣ(最新),ＦＩ(最新),Ｆターム(最新),請求の範囲(独立請求項),"ＩＰＣ(最新,筆頭)",ラベル,インキ,機構,受容シート,用途,0_0,1_1,1_3,1_10,3_1,3_3,3_10,10_1,10_3,10_10
0,特開2019-218636,染色物の製造方法、染色調整インクジェットインク、及び染色調整インクジェットインクセット,株式会社ミマキエンジニアリング,D06P5/00;C09D11/54;C09D11/32;C09D11/40;D06P5/3...,D06P 5/00 120 Z;B41J 2/01 501;B41M 5/...,2C056 FC01;2C056 HA42;2C056 EA04;2C056 FB03;2H...,【請求項１】 染色液によるメディアへの着色度合を調整する単独では前記メディア上で視認困難な...,D06P5/00,1,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,特開2019-218522,インクジェットインク,ゼネラル株式会社,C09D11/30;C09D11/38;B41J2/01;B41M5/00;A61K9/20...,C09D 11/30;B41J 2/01 501;B41M 5/00 120...,2C056 FC01;2H186 BA08;2H186 DA12;2H186 FB11;2H...,【請求項１】 着色剤、アラビアガム、炭素数１２以上、２０以下の高級アルコール、炭素数１以上...,C09D11/30,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,特開2019-218514,インクジェット記録液セット、インクジェット記録用前処理液の製造方法、印刷物及びインクジェット...,コニカミノルタ株式会社,C09D11/54;C09D11/322;B41J2/01;B41M5/00,C09D 11/54;B41J 2/01 123;B41J 2/01 501...,2C056 FC01;2C056 FB02;2C056 EA13;2C056 EA04;2C...,【請求項１】 少なくとも前処理液とインクとからなるインクジェット記録液セットであって、 ...,C09D11/54,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,特開2019-218435,インクジェットインク、インクジェットプリント物およびインクジェットプリント物の製造方法,セーレン株式会社,C09D11/30;B41J2/01;B41M5/00,C09D 11/30;B41J 2/01 501;B41M 5/00 120...,2C056 EA04;2C056 FC01;2H186 AB11;2H186 BA11;2H...,【請求項１】 平均粒子径が０．４～２．５μｍであり、屈折率が１．４～１．７であり、比重が２...,C09D11/30,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,特開2019-217771,インクジェット印刷によって物体に印刷を行う方法,ハイデルベルガードルツクマシーネンアクチエンゲゼルシヤフト,B41J2/01,B41J 2/01 129;B41J 2/01 109;B41J 2/01...,2C056 EA04;2C056 HA44;2C056 FB09,【請求項１】 インクジェット印刷によって物体に印刷を行う方法であって、 以下のステップ：...,B41J2/01,1,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 正解ラベル

サンプルデータの 'インキ', '機構', '受容シート', '用途' の各列に該当する場合1、該当しない場合0の数字を振っていますが、これがラベルです。

サンプルデータでは 1つの文書は4つのカテゴリのどれかに属するようにラベルを振っていますので、ラベルは one-hot ベクトルになっています。

In [37]:
#'インキ', '機構', '受容シート', '用途' の4カテゴリの場合
import numpy as np

y_4category = data[['インキ', '機構', '受容シート', '用途']].values
np.save('data/y_4category.npy', y_4category)

y_4category

array([[0, 1, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       ...,
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [1, 0, 0, 0]])

In [36]:
# {'インキ', '機構', '受容シート'}, {'用途'} の2カテゴリの場合
# 用途 or 用途でない と分類するラベルは、用途が(1, 0)からなるので、
# 1から引く(1-1=0, 1-0=1)ことで用途でないラベルを作ることができる

y_2category = pd.concat([data['用途'], (1-data['用途'])], axis=1).values
np.save('data/y_2category.npy', y_2category)

y_2category

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [1, 0],
       [0, 1],
       [0, 1]])

### 整数のカテゴリからone-hot型のカテゴリを作る場合

In [22]:
print('整数のカテゴリ\n', data['ラベル'].values, '\n')

# 対角行列の作成
import numpy as np
Dmatrix = np.eye(max(data['ラベル'])+1) # ラベルは0-3 の4値なので最大値+1
print('対角行列\n', Dmatrix, '\n')

label = Dmatrix[data['ラベル'].values]
print('one-hot型のカテゴリ\n', label)

整数のカテゴリ
 [1 0 0 ... 3 1 0] 

対角行列
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]] 

one-hot型のカテゴリ
 [[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


### one-hot型のカテゴリから整数のカテゴリを作る場合

In [32]:
one_hot = data[['インキ', '機構', '受容シート', '用途']].values
print('one-hot型のカテゴリ\n', one_hot, '\n')

import numpy as np
label = np.argmax(one_hot, axis=1)
print('整数のカテゴリ\n', label)

one-hot型のカテゴリ
 [[0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 ...
 [0 0 0 1]
 [0 1 0 0]
 [1 0 0 0]] 

整数のカテゴリ
 [1 0 0 ... 3 1 0]


## 学習データと検証データに分割

文書ベクトル（x）とラベル（y）が同じように分割されなければならないので

Excelを読み込んだPandas.DataFrameのインデックスをランダムシャッフルし、4分割して分割用インデックスを作ります。

In [ ]:
# インデックスのランダムシャッフル、4分割
import numpy as np

index = np.random.permutation(data.index)
div_index = [[], [], [], []]
div = len(index)//4
div_index[0] = index[:div]
div_index[1] = index[div:div*2]
div_index[2] = index[div*2:div*3]
div_index[3] = index[div*3:]
div_index = np.array(div_index)
np.save('data/index.npy', div_index)
print(div_index)

[array([2422,  825,  948, 1525, 1118, 1207, 2794,  893, 1743, 3160, 3419,
       1708, 3392, 1904, 2906, 2818,  993, 3556, 2514, 3265, 2641, 1173,
       1903, 1233, 1056,  385, 2539, 3529,  831, 3010, 1785, 2504, 2765,
        461, 1736, 1372, 3183, 1860, 1788, 3497, 3520,  411, 1988, 2125,
        510, 2027,  443, 2142, 2870, 1617, 2326, 2759,  302, 3041, 2498,
       3080,  608,  541, 3108, 1029, 3336, 2513, 1937, 1840,  764,  482,
       3495, 3563,  406, 1835, 2519, 1041,  530, 3141,  653, 2878, 2120,
       2430,  878, 1280,  757, 2844, 3574, 3086, 3440,  489, 2297, 1419,
       1485, 2672,  157,  781,  710,  464,  527, 2945,  478, 3172, 2684,
       1360,  777, 1621, 2993, 2410, 3554, 1232, 2788, 3073, 3050, 1066,
       3192, 1529, 1267, 2271, 1172,  605, 1732, 2890, 1300,  715, 1952,
       1942, 2008,  142, 3460, 2545,  720, 3168, 1368, 3253, 1640, 2475,
       1857,  355,  785, 1251, 1383, 1962, 2373,  701, 2980, 1006, 2732,
        516, 3550, 1907, 2923, 1830,  272, 2246, 1

In [ ]:
from scipy.sparse import csr_matrix
from scipy.io import mmwrite, mmread

x_ = mmread('data/mecab_TfIdf_csr.mtx')
x_ = csr_matrix(x_)
div_index = np.load('data/index.npy', allow_pickle=True)
for i in range(4):
  print('文書ベクトル\n', x_[div_index[i]].todense(), 
        '\n4値ラベル\n', y_4category[div_index[i]], 
        '\n2値ラベル\n', y_2category[div_index[i]], '\n')

文書ベクトル
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
4値ラベル
 [[0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 ...
 [0 1 0 0]
 [0 0 1 0]
 [0 1 0 0]] 
2値ラベル
 [[0 1]
 [0 1]
 [0 1]
 ...
 [0 1]
 [0 1]
 [0 1]] 

文書ベクトル
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
4値ラベル
 [[0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 ...
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]] 
2値ラベル
 [[0 1]
 [0 1]
 [0 1]
 ...
 [0 1]
 [0 1]
 [0 1]] 

文書ベクトル
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
4値ラベル
 [[0 0 0 1]
 [0 1 0 0]
 [0 1 0 0]
 ...
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]] 
2値ラベル
 [[1 0]
 [0 1]
 [0 1]
 ...
 [0 1]
 [0 1]
 [0 1]] 

文書ベクトル
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0

## [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)による2値分類

参考

[[python 機械学習初心者向け] scikit-learnでSVMを簡単に実装する](https://qiita.com/kazuki_hayakawa/items/18b7017da9a6f73eba77)

[SVMによる文書分類とその応用](https://qiita.com/3000manJPY/items/a0652d488ce3c956613d)

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.io import mmread
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# パラメータチューニング用定数
tuned_parameters = [
                    {'C': [0.01, 0.1, 1, 10, 100, 1000], 'kernel': ['linear']},
                    {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.001, 0.0001]},
                    {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['poly'], 'gamma': [0.001, 0.0001], 'degree': [2, 3, 4]},
                    {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['sigmoid'], 'gamma': [0.001, 0.0001]}
                    ]

x_ = mmread('data/mecab_TfIdf_csr.mtx')
x_ = csr_matrix(x_)
y_ = np.load('data/y_2category.npy')
result = []
print('2値分類を開始')
# 初めに先頭から300個のデータでパラメータチューニングする
score = 'f1'
clf = GridSearchCV(
    SVC(), # 識別器
    tuned_parameters, # 最適化したいパラメータセット 
    #cv=0, # 交差検定の回数
    n_jobs = -1, # CPUのコア数(-1で自動決定)
    scoring='%s_weighted' % score,  # モデルの評価関数の指定
    verbose=True)
clf.fit(x_[:300].todense(), y_[:300, 0])

p = clf.best_estimator_
print('\n最適化パラメータ：', p, '\n')

# パラメータチューニングで求めた定数で分類器を生成
clf = SVC(kernel=p.kernel, C=p.C, gamma=p.gamma, degree=p.degree)

# 4分割交差検証
div_index = np.load('data/index.npy', allow_pickle=True)
for i in range(4):
  train_index = []
  for j in range(4):
    if i == j:
      test_index = div_index[j]
    else:
      train_index.append(div_index[j])
  train_index = np.hstack(train_index)

  # 学習
  print(str(i+1)+'回目', '学習')
  clf.fit(x_[train_index].todense(), y_[train_index, 0])

  # 未知データを入力して正答率を求める
  pred_data = clf.predict(x_[test_index].todense())
  accuracy = accuracy_score(pred_data, y_[test_index][:, 0])
  print('未知データ推論，正答率：{0:.3%}'.format(accuracy))
  if p.kernel == 'linear':
    p.gamma = ''
  if p.gamma != 'poly':
    p.degree = ''
  result.append([p.kernel, p.C, p.gamma, p.degree, 'mecab', 'TfIdf', i+1, accuracy])

print('\n', np.array(result)[-4:])

2値分類を開始
Fitting 5 folds for each of 56 candidates, totalling 280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   26.5s
[Parallel(n_jobs=-1)]: Done 280 out of 280 | elapsed:   37.3s finished



最適化パラメータ： SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False) 

1回目 学習
未知データ推論，正答率：98.446%
2回目 学習
未知データ推論，正答率：99.112%
3回目 学習
未知データ推論，正答率：97.558%
4回目 学習
未知データ推論，正答率：98.559%

 [['linear' '10' '' '' 'mecab' 'TfIdf' '1' '0.9844617092119867']
 ['linear' '10' '' '' 'mecab' 'TfIdf' '2' '0.9911209766925638']
 ['linear' '10' '' '' 'mecab' 'TfIdf' '3' '0.9755826859045506']
 ['linear' '10' '' '' 'mecab' 'TfIdf' '4' '0.9855875831485588']]


## [LightGBM](https://lightgbm.readthedocs.io/en/latest/)による4値分類

In [ ]:
import os
from google.colab import drive

# googleドライブのマウント 
drive.mount('/content/drive/')

# 作業ファイルをマイドライブに変更
os.chdir('/content/drive/My Drive/') 

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# GPU版LightGBMのインストール
%cd /content/
!git clone --recursive https://github.com/Microsoft/LightGBM
%cd /content/LightGBM/
!mkdir build
!cmake -DUSE_GPU=1 #avoid ..
!make -j$(nproc)    
!sudo apt-get -y install python-pip
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
%cd /content/LightGBM/python-package
!sudo python setup.py install --precompile

/content
Cloning into 'LightGBM'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 19707 (delta 18), reused 22 (delta 12), pack-reused 19656
Receiving objects: 100% (19707/19707), 15.78 MiB | 5.24 MiB/s, done.
Resolving deltas: 100% (14375/14375), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728), 8.51 MiB | 2.02 MiB/s, done.
Resolving deltas: 100% (17565/17565), done.
Submodule path 'compute': checked out '36c89134d4013b2e5e45bc55656a18bd6141995a'
/content/LightGBM
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C

In [ ]:
import sys
from IPython.display import clear_output
import warnings
warnings.simplefilter('ignore')
import time
from scipy.sparse import csr_matrix
from scipy.io import mmread
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

In [ ]:
## グリッドサーチ不使用

# パラメータ無調整
print('パラメータ無調整')

# ハイパーパラメーターの設定
lgbm_params = {#'task'         : 'train',
          'boosting_type': 'gbdt',
          'device_type'  : 'gpu', 
          'objective'    : 'multiclass', 
          'seed'         : 71, 
          'verbosity'      : -1, 
          'metrics'      : {'multi_logloss'}, 
          'num_class'    : 4, 
          }


x_ = mmread('data/mecab_TfIdf_csr.mtx')
x_ = csr_matrix(x_)
y_ = np.load('data/y_4category.npy')
print('mecab_TfIdfの4値分類を開始')

# 4分割交差検証
div_index = np.load('data/index.npy', allow_pickle=True)
for i in range(4):
  train_index = []
  for j in range(4):
    if i == j:
      test_index = div_index[j]
    else:
      train_index.append(div_index[j])
  train_index = np.hstack(train_index)

  # 上記のパラメータでモデルを学習する
  print(str(i+1)+'回目', '学習')
  lgb_train = lgb.Dataset(x_[train_index].toarray(), label=np.argmax(y_[train_index], axis=1))
  lgb_eval = lgb.Dataset(x_[test_index].toarray(), label=np.argmax(y_[test_index], axis=1))
  model = lgb.train(lgbm_params, lgb_train)

  # テストデータを予測する
  y_calc = model.predict(x_[test_index].toarray())
  y_pred_max = np.argmax(y_calc, axis=1)  # 最尤と判断したクラスの値にする
  y_test_max = np.argmax(y_[test_index], axis=1)

  # 正答率とRMSEを計算
  acc = sum(y_pred_max == y_test_max) / len(y_pred_max)
  RMSE = np.sqrt(mean_squared_error(y_[test_index], y_calc))
  print(' 未知データ推論，正答率：{0:.3%}，誤差RMSE：{1:.5}'.format(acc, RMSE))


パラメータ無調整
mecab_TfIdfの4値分類を開始
1回目 学習
 未知データ推論，正答率：94.895%，誤差RMSE：0.14596
2回目 学習
 未知データ推論，正答率：94.451%，誤差RMSE：0.15605
3回目 学習
 未知データ推論，正答率：93.230%，誤差RMSE：0.16304
4回目 学習
 未知データ推論，正答率：94.346%，誤差RMSE：0.15739


In [ ]:
# ハイパーパラメータ自動最適化ツール
! pip install optuna
%cd /content/drive/My Drive/ 

     |████████████████████████████████| 256kB 2.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163kB 8.4MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 112kB 11.1MB/s 
     |████████████████████████████████| 143kB 8.7MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
  Created wheel for optuna: filename=optuna-2.2.0-cp36-none-any.whl size=340928 sha256=7402788cfebf6b66e48e9d4c1e1923138904c0517d58f370a9477711fe4372a3
  Stored in directory: /root/.cache/pip/wheels/a6/27/26/3ef57a8fb2c8bceef3f4ece73483cec1895e06eae442d34a41
Successfully built optuna
  Created wheel for pyperclip: filename=pyperclip-1.8.0-cp36-none-any.whl size=8693 sha256=6722539059a9c8e76394c87ffa27721223ddf1a6f763340fad7384737e65a97f
  Stored in directory: /root/.cache/pip/wheels/b2/

In [ ]:
# ハイパーパラメータ自動最適化ツール使用
import optuna

def objective(trial):
  # ハイパーパラメーターの設定
  lgbm_params = {'task'         : 'train',
                 'boosting_type': 'gbdt',
                 'device_type'  : 'gpu', 
                 'objective'    : 'multiclass', 
                 'seed'         : 71, 
                 'verbosity'      : -1, 
                 'metrics'      : {'multi_logloss'},
                 #'metrics'      : {'multi_error'}, 
                 'num_class'    : 4, 
                 'max_depth'    : trial.suggest_int('max_depth', 5, 10), 
                 'subsumple'    : trial.suggest_uniform('subsumple', 0.6, 1.0), 
                 'subsample_freq' : trial.suggest_int('subsample_freq', 0, 1), 
                 'leaning_rate' : trial.suggest_loguniform('leaning_rate', 1e-5, 0.5), 
                 'feature_fraction' : trial.suggest_uniform('feature_fraction', 0.0, 1.0), 
                 'lambda_l1'    : trial.suggest_uniform('lambda_l1' , 0.0, 1.0), 
                 'lambda_l2'    : trial.suggest_uniform('lambda_l2' , 0.0, 1.0)
                 }
  # 上記のパラメータでモデルを学習する
  lgb_train = lgb.Dataset(x_[train_index].toarray(), label=np.argmax(y_[train_index], axis=1))
  lgb_eval = lgb.Dataset(x_[test_index].toarray(), label=np.argmax(y_[test_index], axis=1))#, reference=lgb_train)
  model = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval, 
                    num_boost_round=num_round_train, early_stopping_rounds=8)
  clear_output()
  print(result,'\n')
  print('残時間 : '+'{0:.1f}'.format(timeout-(time.time()-start)), '\n')

  # 誤差の算出
  # テストデータを予測する
  y_calc = model.predict(x_[test_index].toarray())
  y_pred_max = np.argmax(y_calc, axis=1)  # 最尤と判断したクラスの値にする
  y_test_max = np.argmax(y_[test_index], axis=1)

  # 正答率とRMSEを計算
  acc = sum(y_pred_max == y_test_max) / len(y_pred_max)
  RMSE = np.sqrt(mean_squared_error(y_[test_index], y_calc))
  val_RMSE.append(RMSE)
  
  # 誤差(RMSE)が小さくなった時にモデルを保存する
  try:
    if val_RMSE[-2] > val_RMSE[-1]:
      print('モデル保存 data/model.txt')
      learning_flug = 0
      model.save_model('data/model.txt')
  except:
    pass

  return RMSE

num_round_train = 300
timeout = 600

result = []
for i in range(4):
  train_index = []
  result.append(str(i+1)+'回目 学習')
  for j in range(4):
    if i == j:
      test_index = div_index[j]
    else:
      train_index.append(div_index[j])
  train_index = np.hstack(train_index)

  # ハイパーパラメーター調整
  start = time.time()
  val_RMSE = []
  study = optuna.create_study()
  study.optimize(objective, timeout=timeout)

  print('Best trial:')
  trial = study.best_trial
  print('Value:{}'.format(trial.value))
  print('Params:')
  for key, value in trial.params.items():
    print('"{}" : {}'.format(key, value), )
  print('\n')

  # 保存したモデルを読み込んで正答率計算
  model = lgb.Booster(model_file='data/model.txt')

  # テストデータを予測する
  y_calc = model.predict(x_[test_index].toarray())
  y_pred_max = np.argmax(y_calc, axis=1)  # 最尤と判断したクラスの値にする
  y_test_max = np.argmax(y_[test_index], axis=1)

  # 正答率とRMSEを計算
  acc = sum(y_pred_max == y_test_max) / len(y_pred_max)
  RMSE = np.sqrt(mean_squared_error(y_[test_index], y_calc))
  print(' 未知データ推論，正答率：{0:.3%}，誤差RMSE：{1:.5}'.format(acc, RMSE), '\n')
  result.append(' 未知データ推論，正答率：{0:.3%}，誤差RMSE：{1:.5}'.format(acc, RMSE))

print()
for text in result:
  print(text)

['1回目 学習', ' 未知データ推論，正答率：94.784%，誤差RMSE：0.14113', '2回目 学習', ' 未知データ推論，正答率：94.340%，誤差RMSE：0.14694', '3回目 学習', ' 未知データ推論，正答率：93.896%，誤差RMSE：0.15253', '4回目 学習'] 

残時間 : -1.6 

モデル保存 data/model.txt


[I 2020-10-09 15:06:15,587] Trial 303 finished with value: 0.14875229573407214 and parameters: {'max_depth': 5, 'subsumple': 0.7993690299084273, 'subsample_freq': 1, 'leaning_rate': 0.00017200765678510287, 'feature_fraction': 0.29321981391748814, 'lambda_l1': 0.18783076819508437, 'lambda_l2': 0.8552424793542265}. Best is trial 166 with value: 0.14770851556690576.


Best trial:
Value:0.14770851556690576
Params:
"max_depth" : 5
"subsumple" : 0.9726270532916703
"subsample_freq" : 1
"leaning_rate" : 0.00020038320887802013
"feature_fraction" : 0.33278198721413355
"lambda_l1" : 0.021921695110118433
"lambda_l2" : 0.3905854445325143


 未知データ推論，正答率：94.568%，誤差RMSE：0.14875 


1回目 学習
 未知データ推論，正答率：94.784%，誤差RMSE：0.14113
2回目 学習
 未知データ推論，正答率：94.340%，誤差RMSE：0.14694
3回目 学習
 未知データ推論，正答率：93.896%，誤差RMSE：0.15253
4回目 学習
 未知データ推論，正答率：94.568%，誤差RMSE：0.14875
